Walkthrough used: 
- https://techs0uls.wordpress.com/2019/12/12/donut-unsupervised-anomaly-detection-using-vae/
- https://github.com/NetManAIOps/donut
- https://github.com/thu-ml/zhusuan
- https://github.com/haowen-xu/tfsnippet

In [4]:
import numpy as np
import pandas as pd
from donut import complete_timestamp, standardize_kpi, Donut, DonutTrainer, DonutPredictor
import tensorflow as tf
from tensorflow import keras as K
from tfsnippet.modules import Sequential
from tfsnippet.utils import get_variables_as_dict, VariableSaver
import tensorflow.compat.v1 as tf
import os
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score
tf.disable_v2_behavior()
import warnings
tf.logging.set_verbosity(tf.logging.ERROR)

In [5]:
data = pd.read_csv(r'C:\Users\Payton Rudnick\Documents\donut\sample_data\g.csv')

In [6]:
data

,timestamp,value,label
0,1476460800,0.012604,0
1,1476460860,0.017786,0
2,1476460920,0.012014,0
3,1476460980,0.017062,0
4,1476461040,0.023632,0
...,...,...,...
214879,1489420500,0.024097,0
214880,1489420560,0.009231,0
214881,1489420620,0.008676,0
214882,1489420680,0.029228,0


In [7]:
#Seems like we have 209 'anomalies' and 17,359 'normal' datapoints
print(data['label'].value_counts())

0    198771
1     16113
Name: label, dtype: int64


In [8]:
timestamp = data['timestamp']
values = data['value']
labels = data['label']

In [9]:
print("Timestamps: {}".format(timestamp.shape[0]))

Timestamps: 214884


In [10]:
#Complete the timestamp, and obtain the missing point indicators

timestamp, missing, (values, labels) = complete_timestamp(timestamp, (values, labels))

print("Missing Points: {}".format(np.sum(missing == 1)))
print("Labeled Anomalies: {}".format(np.sum(labels == 1)))

Missing Points: 1116
Labeled Anomalies: 16113


In [11]:
#Spliting the train & test dataset 

#Setting the test portion of the dataset 
test_portion = 0.3

test_n = int(len(values) * test_portion)

train_values, test_values = values[:-test_n], values[-test_n:]

train_labels, test_labels = labels[:-test_n], labels[-test_n:]

train_missing, test_missing = missing[:-test_n], missing[-test_n:]

print("Rows in test set: {}".format(test_values.shape[0]))
print("Anomalies in test set: {}".format(np.sum(test_labels == 1)))

Rows in test set: 64800
Anomalies in test set: 6334


There are currently 6,334 anomaly points in the test set out of 64,800 data points. Is around 9.77% which is way too many but that is a problem for later

In [12]:
#Standardize the training and testing data 

train_values, mean, std = standardize_kpi(train_values, excludes=np.logical_or(train_labels, train_missing))

In [13]:
test_values, _, _ = standardize_kpi(test_values, mean=mean, std=std)

print("Train values mean: {}".format(mean))
print("Train values std: {}".format(std))

Train values mean: 0.04438549280166626
Train values std: 0.04308837652206421


We will build the entire model within the scope of 'model_vs', it should hold exactly all the variables of 'model', including the variables created by the Keras layers 

In [14]:
#Defining the sliding window
sliding_window = 120

In [15]:
with tf.variable_scope('model') as model_vs:
    model = Donut(
        h_for_p_x=Sequential([
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
        ]),
        h_for_q_z=Sequential([
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
        ]),
        x_dims=sliding_window,
        z_dims=5,
    )

Using it for prediction: 

In [16]:
#Using the DonutTrainer class to train the model 
trainer = DonutTrainer(model=model, model_vs=model_vs, max_epoch=30)

predictor = DonutPredictor(model)

with tf.Session().as_default(): 
    trainer.fit(train_values, train_labels, train_missing, mean, std)
    #Making predictions
    test_score = predictor.get_score(test_values, test_missing)
    print("Number of predictions: {}".format(test_score.shape[0]))
    # try different thresholds
    best_threshold = 0
    best_f1 = 0
    best_predictions = []
    thresholds = np.arange(5, 50, 0.2)
    for t in thresholds:
        threshold = t  # can be changed to better fit the training data
        anomaly_predictions = []
        for l in test_score:
            if abs(l) > threshold:
                anomaly_predictions.append(1)
            else:
                anomaly_predictions.append(0)
        for i in range(sliding_window-1, len(anomaly_predictions)):
            if anomaly_predictions[i-sliding_window+1] == 1 and test_labels[i] == 1:  # true positive
                j = i-1
                while j >= sliding_window-1 and test_labels[j] == 1\
                        and anomaly_predictions[j-sliding_window+1] == 0:
                    anomaly_predictions[j-sliding_window+1] = 1
                    j -= 1
                j = i+1
                while j < len(anomaly_predictions) and test_labels[j] == 1\
                        and anomaly_predictions[j-sliding_window+1] == 0:
                    anomaly_predictions[j-sliding_window+1] = 1
                    j += 1
        f1 = f1_score(test_labels[sliding_window-1:], anomaly_predictions, average='binary')
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
            best_predictions = anomaly_predictions

    anomaly_predictions = np.array(best_predictions)
    print("-- Final Results --")
    print("Best anomaly threshold {}".format(best_threshold))
    print("Anomalies found: {}/{}".format(np.sum(anomaly_predictions == 1), np.sum(test_labels == 1)))
    prfs = precision_recall_fscore_support(test_labels[sliding_window-1:], anomaly_predictions)
    print("-- Anomaly Rows --")
    print("Precision: {:.3f}".format(prfs[0][1]))
    print("Recall: {:.3f}".format(prfs[1][1]))
    print("F1-score: {:.3f}".format(prfs[2][1]))

Trainable Parameters                     (58,150 in total)
----------------------------------------------------------
donut/p_x_given_z/x_mean/bias           (120,)         120
donut/p_x_given_z/x_mean/kernel         (100, 120)  12,000
donut/p_x_given_z/x_std/bias            (120,)         120
donut/p_x_given_z/x_std/kernel          (100, 120)  12,000
donut/q_z_given_x/z_mean/bias           (5,)             5
donut/q_z_given_x/z_mean/kernel         (100, 5)       500
donut/q_z_given_x/z_std/bias            (5,)             5
donut/q_z_given_x/z_std/kernel          (100, 5)       500
sequential/forward/_0/dense/bias        (100,)         100
sequential/forward/_0/dense/kernel      (5, 100)       500
sequential/forward/_1/dense_1/bias      (100,)         100
sequential/forward/_1/dense_1/kernel    (100, 100)  10,000
sequential_1/forward/_0/dense_2/bias    (100,)         100
sequential_1/forward/_0/dense_2/kernel  (120, 100)  12,000
sequential_1/forward/_1/dense_3/bias    (100,)         1